In [ ]:
import sys
print(sys.path)

In [ ]:
sys.path.append('/content/drive/MyDrive/Python/packages/')

In [ ]:
print(sys.path)

In [ ]:
import ipywidgets as widgets

In [ ]:
from IPython.display import display,clear_output

In [ ]:
text_input = widgets.Text(
    value='',
    description='Name The Molecule:',
    disabled=False
)

In [ ]:
def on_submit(sender):
    print(f"Molecule : {sender.value}")

In [ ]:
text_input.on_submit(on_submit)

In [ ]:
display(text_input)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
numer_atoms = widgets.IntText(
    value=1,
    description='Number of Atoms',
    disabled=False
)

atom_box = widgets.VBox([])

def create_atom_inputs(change):
    clear_output(wait=True)
    display(numer_atoms, atom_box, submit_button)
    n = change['new']
    atom_inputs = []
    for i in range(n):
        atom_symbol = widgets.Text(
            description=f'Atom {i+1}:',
            placeholder='e.g., H, O, C'
        )
        x = widgets.FloatText(description='x:')
        y = widgets.FloatText(description='y:')
        z = widgets.FloatText(description='z:')
        box = widgets.HBox([atom_symbol, x, y, z])
        atom_inputs.append(box)
    atom_box.children = atom_inputs

def on_submit_clicked(b):
    geo = []
    for box in atom_box.children:
        atom_symbol = box.children[0].value.strip()
        x = box.children[1].value
        y = box.children[2].value
        z = box.children[3].value
        if atom_symbol:
            geo.append((atom_symbol, (x, y, z)))

    print("\nSuccess! Molecule data received:")
    for atom in geo:
        print(atom)

    global molecule_geo
    molecule_geo = geo

numer_atoms.observe(create_atom_inputs, names='value')
submit_button = widgets.Button(description='Molecule Received', button_style='success')
submit_button.on_click(on_submit_clicked)

display(numer_atoms, atom_box, submit_button)

In [ ]:
from pyscf import gto, scf

In [ ]:
def get_multiplicity_pyscf(molecule_geo, charge=0, basis='6-31g'):
    mol_str = ""
    for atom, coords in molecule_geo:
        mol_str += f"{atom} {coords[0]} {coords[1]} {coords[2]}\n"
    mol = gto.Mole()
    mol.atom = mol_str
    mol.basis = basis
    mol.charge = charge
    mol.spin = 0
    mol.build()
    spin_elec_diff = mol.nelec[0] - mol.nelec[1] if hasattr(mol, 'nelec') else 0
    total_spin = abs(spin_elec_diff) / 2
    multiplicity = int(2 * total_spin + 1)
    print("Molecule Geometry:")
    for atom in molecule_geo:
        print(atom)
    print(f"\nCharge: {charge}")
    print(f"Estimated multiplicity: {multiplicity}")

    return multiplicity

In [ ]:
from openfermion.chem import MolecularData

In [ ]:
basis = '6-31g'
charge = 0
multiplicity = get_multiplicity_pyscf(molecule_geo, charge, basis)
molecule = MolecularData(molecule_geo,basis,multiplicity,charge)
print(molecule_geo)

In [ ]:
from openfermion import MolecularData, get_fermion_operator, jordan_wigner
from openfermionpyscf import run_pyscf

In [ ]:
molecule = MolecularData(geometry=molecule_geo,
                         basis=basis,
                         multiplicity=multiplicity,
                         charge=charge)

In [ ]:
molecule = run_pyscf(molecule, run_scf=True, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [ ]:
fermionic_hamiltonian = molecule.get_molecular_hamiltonian()
fermion_op = get_fermion_operator(fermionic_hamiltonian)
qubit_op = jordan_wigner(fermion_op)

In [ ]:
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=[],
    active_indices=[0,1]
)

In [ ]:
from openfermion.transforms import jordan_wigner, get_fermion_operator
from openfermion.linalg import get_sparse_operator

In [ ]:
fermion_op = get_fermion_operator(molecular_hamiltonian)
print("Fermionic Hamiltonian:\n", fermion_op)

In [ ]:
jw_op = jordan_wigner(fermion_op)
print("\nJordan-Wigner Transformed Hamiltonian:\n", jw_op)

In [ ]:
import numpy as np
import cirq
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator
from scipy.optimize import minimize
from scipy.sparse.linalg import eigsh

In [ ]:
H_matrix = get_sparse_operator(jw_op).todense()
true_energy, _ = eigsh(H_matrix, k=1, which='SA')
print(f" Exact Ground State Energy (reference): {true_energy[0]:.6f}")

In [ ]:
n_qubits = int(np.log2(H_matrix.shape[0]))
print("Hamiltonian needs", n_qubits, "qubits")

In [ ]:
def create_ansatz(params, n_qubits, num_layers=3):
    qubits = [cirq.LineQubit(i) for i in range(n_qubits)]
    circuit = cirq.Circuit()
    required_params = n_qubits * num_layers * 4
    if len(params) != required_params:
        raise ValueError(f"Expected {required_params} parameters, but got {len(params)}")
    param_idx = 0
    for layer in range(num_layers):
        for q in qubits:
            circuit.append(cirq.rx(params[param_idx])(q)); param_idx += 1
            circuit.append(cirq.ry(params[param_idx])(q)); param_idx += 1
            circuit.append(cirq.rz(params[param_idx])(q)); param_idx += 1
        for i in range(n_qubits - 1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
        for q in qubits:
            circuit.append(cirq.rz(params[param_idx])(q)); param_idx += 1
    return circuit, qubits

In [ ]:
def energy_expectation(params):
    circuit, qubits = create_ansatz(params, n_qubits)
    simulator = cirq.Simulator()
    result = simulator.simulate(circuit)
    psi = result.final_state_vector.reshape(-1, 1)
    energy = np.real((psi.conj().T @ H_matrix @ psi)[0, 0])
    return energy

In [ ]:
num_layers = 3
required_param_count = n_qubits * num_layers * 4
initial_params = np.random.uniform(0, 2*np.pi, required_param_count)
res = minimize(energy_expectation, initial_params, method='COBYLA')
true_energy = float(true_energy[0])
print("\nOptimization complete")
print(f"Estimated Ground State Energy (VQE): {res.fun:.6f}")
print(f"Reference Energy: {true_energy:.6f}")
print(f"Error: {abs(res.fun - true_energy):.6e}")

In [ ]:
from scipy.sparse.linalg import eigsh
_, vecs = eigsh(H_matrix, k=1, which='SA')
exact_state = vecs[:, 0].reshape(-1, 1)
circuit, qubits = create_ansatz(res.x,n_qubits)
simulator = cirq.Simulator()
vqe_state = simulator.simulate(circuit).final_state_vector.reshape(-1, 1)
fidelity = np.abs(np.vdot(exact_state, vqe_state))**2
print(f"🔹 Fidelity |⟨exact|vqe⟩|² = {fidelity:.6f}")

In [ ]:
import numpy as np, cirq
from scipy.linalg import expm
from collections import Counter

t_candidates = [0.05, 0.1, 0.2, 0.4, 1.0,1.5,2.0]
m_candidates = [6, 8, 10,12,14]
reps = 4096

def inverse_qft_on(qubits):
    circ = cirq.Circuit()
    n = len(qubits)
    for j in range(n):
        k = n - j - 1
        for i in range(k+1, n):
            power = i - k
            angle = -np.pi / (2**power)
            circ.append(cirq.CZ(qubits[i], qubits[k])**(angle/np.pi))
        circ.append(cirq.H(qubits[k]))
    for i in range(n // 2):
        circ.append(cirq.SWAP(qubits[i], qubits[n - i - 1]))
    return circ

n_qubits = int(np.log2(H_matrix.shape[0]))
param_len = len(res.x)
num_layers_used = param_len // (n_qubits * 4)

best_overall = {"abs_error": np.inf}
for t in t_candidates:
    for m in m_candidates:
        count_qubits = [cirq.LineQubit(i) for i in range(m)]
        target_qubits = [cirq.LineQubit(i + m) for i in range(n_qubits)]
        try:
            ansatz_circ, _ = create_ansatz(res.x, n_qubits, num_layers=num_layers_used)
        except Exception as e:
            continue
        def remap(q):
            return cirq.LineQubit(q.x + m) if isinstance(q, cirq.LineQubit) else q
        ansatz_on_targets = ansatz_circ.transform_qubits(remap)
        circuit = cirq.Circuit()
        circuit += ansatz_on_targets
        circuit.append([cirq.H(q) for q in count_qubits])
        for k in range(m):
            exp_factor = 2**k
            U_pow = expm(-1j * H_matrix * t * exp_factor)
            U_gate = cirq.MatrixGate(U_pow)
            controlled = U_gate.controlled()
            control = count_qubits[m - 1 - k]
            circuit.append(controlled.on(control, *target_qubits))
        circuit += inverse_qft_on(count_qubits)
        circuit.append(cirq.measure(*count_qubits, key='phase'))
        sim = cirq.Simulator()
        result = sim.run(circuit, repetitions=reps)
        hist = result.histogram(key='phase')
        if not hist:
            continue
        peaks = Counter(hist).most_common(8)
        for bit_int, cnt in peaks:
            bin_str = format(bit_int, f'0{m}b')
            decimal_phase = int(bin_str, 2) / (2**m)
            energy_raw = -2 * np.pi * decimal_phase / t
            candidates = []
            for kshift in range(-4, 5):
                energy_unwrapped = energy_raw + kshift * (2 * np.pi / t)
                candidates.append(energy_unwrapped)
            candidates = np.array(candidates)
            try:
                true_e = float(true_energy)
            except:
                true_e = float(np.array(true_energy).flatten()[0])
            idx = np.argmin(np.abs(candidates - true_e))
            energy_best = candidates[idx]
            abs_err = abs(energy_best - true_e)
            record = {"m": m, "t": t, "bitstring": bin_str, "counts": cnt, "energy_est": energy_best, "abs_error": abs_err}
            if abs_err < best_overall["abs_error"]:
                best_overall = record
        if best_overall["abs_error"] < 1e-3:
            break
    if best_overall["abs_error"] < 1e-3:
        break

print(best_overall["m"], best_overall["t"])
print(best_overall["bitstring"])
print(best_overall["energy_est"])
try:
    print(float(true_energy))
except:
    print(float(np.array(true_energy).flatten()[0]))
print(best_overall["abs_error"])

In [ ]:
import numpy as np, cirq
from scipy.linalg import expm
from collections import Counter

t_grid = [0.05, 0.1, 0.2, 0.4, 1.0,1.5,2.0]
m_list = [6, 8, 10,12,14]
reps = 4096

best = {"abs_error": np.inf}
for t in t_grid:
    for m in m_list:
        count_qubits = [cirq.LineQubit(i) for i in range(m)]
        target_qubits = [cirq.LineQubit(i + m) for i in range(n_qubits)]
        ansatz_circ, _ = create_ansatz(res.x, n_qubits, num_layers=(len(res.x)//(n_qubits*4)))
        ansatz_on_targets = ansatz_circ.transform_qubits(lambda q: cirq.LineQubit(q.x + m))
        def inverse_qft_on(qubits):
            circ = cirq.Circuit()
            n = len(qubits)
            for j in range(n):
                k = n - j - 1
                for i in range(k+1, n):
                    angle = -np.pi / (2**(i-k))
                    circ.append(cirq.CZ(qubits[i], qubits[k])**(angle/np.pi))
                circ.append(cirq.H(qubits[k]))
            for i in range(n//2):
                circ.append(cirq.SWAP(qubits[i], qubits[n - i - 1]))
            return circ
        circuit = cirq.Circuit()
        circuit += ansatz_on_targets
        circuit.append([cirq.H(q) for q in count_qubits])
        for k in range(m):
            U_pow = expm(-1j * H_matrix * t * (2**k))
            U_gate = cirq.MatrixGate(U_pow)
            circuit.append(U_gate.controlled().on(count_qubits[m-1-k], *target_qubits))
        circuit += inverse_qft_on(count_qubits)
        circuit.append(cirq.measure(*count_qubits, key='phase'))
        sim = cirq.Simulator()
        result = sim.run(circuit, repetitions=reps)
        hist = result.histogram(key='phase')
        if not hist:
            continue
        peaks = Counter(hist).most_common(5)
        true_e = float(true_energy)
        for bit_int, cnt in peaks:
            bin_str = format(bit_int, f'0{m}b')
            decimal_phase = int(bin_str, 2) / (2**m)
            energy_raw = -2*np.pi*decimal_phase / t
            candidates = energy_raw + np.arange(-8,9)*(2*np.pi/t)
            energy_best = candidates[np.argmin(np.abs(candidates - true_e))]
            abs_err = abs(energy_best - true_e)
            if abs_err < best["abs_error"]:
                best = {"m": m, "t": t, "bitstring": bin_str, "energy_est": energy_best, "abs_error": abs_err, "counts": cnt}
print(best)

In [ ]:
import numpy as np, cirq
from scipy.linalg import expm
from collections import Counter

t_grid = [0.05, 0.1, 0.2, 0.4, 1.0,1.5,2.0]
m_list = [6, 8, 10,12,14]
reps = 4096

best = {
    "abs_error": np.inf,
    "accuracy_rate": 0
}
true_e = float(true_energy)
all_results = []

for t in t_grid:
    for m in m_list:
        count_qubits = [cirq.LineQubit(i) for i in range(m)]
        target_qubits = [cirq.LineQubit(i + m) for i in range(n_qubits)]
        ansatz_circ, _ = create_ansatz(res.x, n_qubits, num_layers=(len(res.x)//(n_qubits*4)))
        ansatz_on_targets = ansatz_circ.transform_qubits(lambda q: cirq.LineQubit(q.x + m))

        def inverse_qft_on(qubits):
            circ = cirq.Circuit()
            n = len(qubits)
            for j in range(n):
                k = n - j - 1
                for i in range(k+1, n):
                    angle = -np.pi / (2**(i-k))
                    circ.append(cirq.CZ(qubits[i], qubits[k])**(angle/np.pi))
                circ.append(cirq.H(qubits[k]))
            for i in range(n//2):
                circ.append(cirq.SWAP(qubits[i], qubits[n - i - 1]))
            return circ

        circuit = cirq.Circuit()
        circuit += ansatz_on_targets
        circuit.append([cirq.H(q) for q in count_qubits])

        for k in range(m):
            U_pow = expm(-1j * H_matrix * t * (2**k))
            U_gate = cirq.MatrixGate(U_pow)
            circuit.append(U_gate.controlled().on(count_qubits[m - 1 - k], *target_qubits))

        circuit += inverse_qft_on(count_qubits)
        circuit.append(cirq.measure(*count_qubits, key='phase'))
        sim = cirq.Simulator()
        result = sim.run(circuit, repetitions=reps)
        hist = result.histogram(key='phase')

        if not hist:
            continue

        total_counts = sum(hist.values())
        peaks = Counter(hist).most_common(5)
        for bit_int, cnt in peaks:
            bin_str = format(bit_int, f'0{m}b')
            decimal_phase = int(bin_str, 2) / (2**m)
            energy_raw = -2 * np.pi * decimal_phase / t
            candidates = energy_raw + np.arange(-8, 9) * (2 * np.pi / t)
            energy_best = candidates[np.argmin(np.abs(candidates - true_e))]

            abs_err = abs(energy_best - true_e)
            percent_err = (abs_err / abs(true_e)) * 100
            accuracy_rate = (cnt / total_counts) * 100

            if abs_err < best["abs_error"]:
                best = {
                    "m": m,
                    "t": t,
                    "bitstring": bin_str,
                    "energy_est": float(energy_best),
                    "true_energy": true_e,
                    "abs_error": float(abs_err),
                    "percent_error": float(percent_err),
                    "counts": cnt,
                    "accuracy_rate": float(accuracy_rate)
                }
                all_results.append(abs_err)

            if abs_err < best["abs_error"]:
                best = abs_err

with open("qpe_B_result.pkl", "wb") as f:
    pickle.dump(all_results, f)

print("===  QPE Parameter Result ===")
for k,v in best.items():
    print(f"{k:20}: {v}")
print("\n=== QPE Parameter Result ===")
print(f"Counting qubits (m):     {best['m']}")
print(f"Evolution time (t):      {best['t']}")
print(f"Measured bitstring:      {best['bitstring']}")
print(f"Counts:                  {best['counts']}")
print(f"Accuracy rate:           {best['accuracy_rate']:.2f}%")
print(f"True energy:             {best['true_energy']:.6f}")
print(f"Estimated energy:        {best['energy_est']:.6f}")
print(f"Absolute error:          {best['abs_error']:.6f}")
print(f"Percentage error:        {best['percent_error']:.4f}%")

In [ ]:
from scipy.sparse.linalg import eigsh
eigvals, eigvecs = eigsh(H_matrix, k=1, which='SA')
exact_state = eigvecs[:,0]
sim = cirq.Simulator()
ansatz_circ, _ = create_ansatz(res.x, n_qubits, num_layers=(len(res.x)//(n_qubits*4)))
vqe_state = sim.simulate(ansatz_circ).final_state_vector
fidelity = np.abs(np.dot(np.conj(exact_state), vqe_state))**2
print(f"Fidelity |⟨exact|VQE⟩|² = {fidelity:.6f}")